In [15]:
from OSMPythonTools.nominatim import Nominatim

from OSMPythonTools.overpass import Overpass
from OSMPythonTools.overpass import overpassQueryBuilder

In [73]:
nominatim = Nominatim()
city = nominatim.query('Auvergne Rhône-Alpes')

In [75]:
selector = '"aeroway"="navigationaid"'
query = overpassQueryBuilder(area=city, elementType='node', selector=selector)

overpass = Overpass()
result = overpass.query(query)

In [70]:
result.toJSON()

{'version': 0.6,
 'generator': 'Overpass API 0.7.59 e21c39fe',
 'osm3s': {'timestamp_osm_base': '2022-12-04T01:24:15Z',
  'timestamp_areas_base': '2022-12-03T22:42:54Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 2401770972,
   'lat': 49.0199012,
   'lon': 2.5001166,
   'tags': {'aeroway': 'navigationaid',
    'airmark': 'beacon',
    'beacon:channel': '100Y',
    'beacon:code': 'CGN',
    'beacon:frequency': '115.35 MHz',
    'beacon:type': 'VOR/DME',
    'name': 'Charles de Gaulle',
    'navigationaid': 'vordme'}},
  {'type': 'node',
   'id': 2401770973,
   'lat': 49.0214098,
   'lon': 2.519308,
   'tags': {'aeroway': 'navigationaid',
    'name': 'PAPI 09R',
    'navigationaid': 'papi'}},
  {'type': 'node',
   'id': 2401770974,
   'lat': 49.0255471,
   'lon': 2.5305093,
   'tags': {'aeroway': 'navigationaid',
    'name': 'PAPI 09L',
    'navigationaid': 'papi'}},
 

In [62]:
for element in elements:
    tags = element.tags()
    if(tags):
        print(tags)